# Multivariate Statistics (Decoding / MVPA) on MEG/EEG Data

In [ ]:
%matplotlib qt

In [ ]:
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import mne

matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

## Load epochs

In [ ]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs.apply_baseline((None, 0))
epochs

## Select epochs of interest

Here, we intend to analyze the auditory epochs.

In [ ]:
epochs_auditory = epochs['Auditory']
epochs_auditory

## Calculate empirical evoked difference

In [ ]:
evoked_diff = mne.combine_evoked(
    [epochs_auditory['Auditory/Left'].average(),
     epochs_auditory['Auditory/Right'].average()],
    weights=[1, -1]  # Subtraction
)

evoked_diff.plot(gfp=True)
mne.viz.plot_compare_evokeds(
    [epochs_auditory['Auditory/Left'].average(),
     epochs_auditory['Auditory/Right'].average(),
     evoked_diff]
)

Okay, but… we want more than that! Let's do some machine learning.

## Equalize the number of epochs

To keep chance level at 50% accuracy, we first equalize the number of epochs in each condition.

In [ ]:
epochs_auditory.equalize_event_counts(epochs_auditory.event_id)
epochs_auditory

## Create input `X` and response `y`.

A classifier takes as input a matrix `X` and returns a vector `y` (consisting of `0` and `1`). Here `X` will be the **data at one time point on all gradiometers** (hence the term multivariate). We want to train our model to discriminate between the  `Auditory/Left` and the `Auditory/Right` trials.

We work with all sensors jointly and try to find a discriminative pattern between the two conditions to predict the experimental condition of individual trials.

For classification we will use the `scikit-learn` package (http://scikit-learn.org/) and MNE-Python functions.

Let's first create the response vector, `y`.

In [ ]:
import numpy as np

# Create an vector with length = no. of trials.
y = np.empty(len(epochs_auditory.events), dtype=int)  

# Which trials are LEFT, which are RIGHT?
idx_left = epochs_auditory.events[:, 2] == epochs_auditory.event_id['Auditory/Left']
idx_right = epochs_auditory.events[:, 2] == epochs_auditory.event_id['Auditory/Right']

# Encode: LEFT = 0, RIGHT = 1.
y[idx_left] = 0
y[idx_right] = 1

print(y)
print(f'\nSize of y: {y.size}')

Now, let's create the input matrix, `X`.

We wish to focus only on the gradiometer channels here, so we use
`pick_types(meg='grad')`. For magnetometer channels, we would need to
pass `meg='mag'`; and for EEG channels: `meg=False, eeg=True`.
We create a copy of the epochs because `pick_types()` operates in-place,
but we would like to keep the original epochs object untouched.


In [ ]:
epochs_auditory_grad = epochs_auditory.copy().pick_types(meg='grad')

# Retrieve the data as a NumPy array.
# The array has the shape: (n_trials, n_channels, n_timepoints)
data = epochs_auditory_grad.get_data()
print(data.shape)

We're almost there! We need to reshape the array such that for each trial, we have a vector `[channel_1_time_1, channel_1_time_2, ..., channel_m_time_n]`, i.e., we aim to reshape `X` to the dimension `(n_trials, n_channels * n_timepoints)`.

In [ ]:
n_trials = data.shape[0]

X = data.reshape(n_trials, -1)
print(X.shape)

### Create a classifier!

We will use plain `scikit-learn` machinery for the first round of classifications. This is to demonstrate that you can simply feed pre-processed data from MNE into `scikit-learn`.

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


# The classifier pipeline: it is extremely important to scale the data
# before running the actual classifier (logistic regression in our case).
clf = make_pipeline(StandardScaler(),
                    LogisticRegression())

# Run cross-validation.
# CV without shuffling – "block cross-validation" – is what we want here
# (scikit-learn doesn't shuffle by default, which is good for us).
n_splits = 5
scoring = 'roc_auc'
cv = StratifiedKFold(n_splits=n_splits)
scores = cross_val_score(clf, X=X, y=y, cv=cv, scoring=scoring)

# Mean and standard deviation of ROC AUC across cross-validation runs.
roc_auc_mean = round(np.mean(scores), 3)
roc_auc_std = round(np.std(scores), 3)

print(f'CV scores: {scores}')
print(f'Mean ROC AUC = {roc_auc_mean:.3f} (SD = {roc_auc_std:.3f})')

### Visualize the cross-validation results.


In [ ]:
fig, ax = plt.subplots()
ax.boxplot(scores,
           showmeans=True, # Green triangle marks the mean.
           whis=(0, 100),  # Whiskers span the entire range of the data.
           labels=['Left vs Right'])
ax.set_ylabel('Score')
ax.set_title('Cross-Validation Scores')

## We can do this more simply using the `mne.decoding` module! Let's go. 🚀

In [ ]:
from sklearn.pipeline import make_pipeline
from mne.decoding import Scaler, Vectorizer, cross_val_multiscore

# First, create X and y.
epochs_auditory_grad = epochs_auditory.copy().pick_types(meg='grad')
X = epochs_auditory_grad.get_data()
y = epochs_auditory_grad.events[:, 2]

# Classifier pipeline.
clf = make_pipeline(
    # An MNE scaler that correctly handles different channel types –
    # isn't that great?!
    Scaler(epochs_grad.info),
    # Remember this annoying and error-prone NumPy array reshaping we had to do
    # earlier? Not anymore, thanks to the MNE vectorizer!
    Vectorizer(),
    # And, finally, the actual classifier.
    LogisticRegression())

# Run cross-validation.
# Note that we're using MNE's cross_val_multiscore() here, not scikit-learn's
# cross_val_score() as above. We simply pass the number of desired CV splits,
# and MNE will automatically do the rest for us.
n_splits = 5
scoring = 'roc_auc'
scores = cross_val_multiscore(clf, X, y, cv=5, scoring='roc_auc')

# Mean and standard deviation of ROC AUC across cross-validation runs.
roc_auc_mean = round(np.mean(scores), 3)
roc_auc_std = round(np.std(scores), 3)

print(f'CV scores: {scores}')
print(f'Mean ROC AUC = {roc_auc_mean:.3f} (SD = {roc_auc_std:.3f})')

## Decoding over time: Comparisons at every single time point.

In the previous examples, we have trained a classifier to discriminate between experimental conditions by using the spatio-temporal patterns of **entire trials**. Consequently, the classifier was (hopefully!) able to predict which activation patterns belonged to which condition. 

However, an interesting neuroscientific is: **Exactly *when* do the brain signals for two conditions differ?**

We can try to answer this question by fitting a classifier **at every single time point.** If the classifier can successfully discriminate between the two conditions, we can conclude that the spatial activation patterns measured by the MEG or EEG sensors differed **at this very time point**.

In [ ]:
from sklearn.preprocessing import StandardScaler
from mne.decoding import SlidingEstimator

# First, create X and y.
epochs_auditory_grad = epochs_auditory.copy().pick_types(meg='grad')
X = epochs_auditory_grad.get_data()
y = epochs_auditory_grad.events[:, 2]

# Classifier pipeline. No need for vectorization as in the previous example.
clf = make_pipeline(StandardScaler(),
                    LogisticRegression())

# The "sliding estimator" will train the classifier at each time point.
scoring = 'roc_auc'
time_decoder = SlidingEstimator(clf, scoring=scoring, n_jobs=1, verbose=True)

# Run cross-validation.
n_splits = 5
scores = cross_val_multiscore(time_decoder, X, y, cv=5, n_jobs=1)

# Mean scores across cross-validation splits, for each time point.
mean_scores = np.mean(scores, axis=0)

# Mean score across all time points.
mean_across_all_times = round(np.mean(scores), 3)
print(f'\n=> Mean CV score across all time points: {mean_across_all_times:.3f}')

### Plot the classification results!

In [ ]:
fig, ax = plt.subplots()

ax.axhline(0.5, color='k', linestyle='--', label='chance')  # AUC = 0.5
ax.axvline(0, color='k', linestyle='-')  # Mark time point zero.
ax.plot(epochs.times, mean_scores, label='score')

ax.set_xlabel('Time (s)')
ax.set_ylabel('Mean ROC AUC')
ax.legend()
ax.set_title('Left vs Right')
fig.suptitle('Sensor Space Decoding')